In [1]:
import pandas as pd
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import everygrams
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk import bigrams
import itertools

from collections import Counter, defaultdict
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import random
import nltk
import cld2


# Modeling - method one

In [2]:
with open("WIKI2019_2.txt") as f:
    all_sents = [line for line in f]

In [ ]:
for n, i in enumerate(all_sents[:20000]):
    with open("./wiki_txt_training/file{}.txt".format(n),"a+") as f:
        f.write(i)
for n, i in enumerate(all_sents[20000:]):
    with open("./wiki_txt_testing/file{}.txt".format(n),"a+") as f:
        f.write(i)

In [183]:
from nltk.corpus import PlaintextCorpusReader
corpus_root = './wiki_txt_training/'
wolof_corpus = PlaintextCorpusReader(corpus_root,'.*')

In [184]:
wolof_corpus.sents()

[['Albert', 'Einstein', 'juddoo', 'Ulma', 'Almaañ', 'ci', 'atum'], ['faatoo', 'Princeton', 'Diiwaan', 'yu', 'Bennoo', 'yu', 'Amerig', 'ci', 'doonoon', 'na', 'ab', 'boroom', 'xamxamu', 'jëmm'], ...]

In [190]:
wolof_model = defaultdict(lambda: defaultdict(lambda: 0))
 
for sentence in wolof_corpus.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        wolof_model[(w1, w2)][w3] += 1

# transform the counts to probabilities
for w1_w2 in wolof_model:
    total_count = sum(wolof_model[w1_w2].values())
    for w3 in wolof_model[w1_w2]:
        wolof_model[w1_w2][w3] /= total_count

In [191]:
# get some example
# sentences generated by the model
ex = []
for i in range(10):
    text = [None, None]
    sentence_finished = False
    while not sentence_finished:
        r = random.random()
        accumulator = .0
        for word in wolof_model[tuple(text[-2:])].keys():
            accumulator += wolof_model[tuple(text[-2:])][word]
            if accumulator >= r:
                text.append(word)
                break
        if text[-2:]==[None, None]:
            sentence_finished = True
    alltext = ' '.join([t for t in text if t])
    ex.append(alltext)

In [192]:
ex

['mu sos ngir dëkksi ci taax yi ak ñu dul ñoom',
 'sóobu na ci yooyu',
 'ànd ak ñoom te nu gën koo jëme kanam',
 'amoon tamit ag joor ak ab coonaam fa ñatti at yu mujj yi daa lijjanti aka nos diggante jaam bi nekk ci sancoom yi nekk ci digg yu yaxantu',
 'moom mi leen may ngeen àgg fa njëkk a taxaw ci di leen tegi coona',
 'da leen daa gàddaayloo',
 'Moom mbootaay gi nag mi ngi ci digg Afrig atum 1876g',
 'Sàmm ag jur',
 'don te mengo tul ak',
 'ña ca gëm ñu ag ragal manu def la ca Baabul Mandab ak barsaqu Suwes']

In [194]:
# guess the third words 

In [193]:
wolof_model['da', 'leen'].keys()

dict_keys(['daa', 'a', 'na', 'di'])

# Modeling - method 2

In [195]:
# use nltk.ml

In [197]:
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import everygrams
from nltk.lm.preprocessing import flatten

In [198]:
# from github repo WOL.txt
with open("WOL.txt") as f:
    wolof = f.read().splitlines() 

with open("wo_SN-web_crawled.txt") as f:
    wol_web = f.read().splitlines() 

with open("WIKI2019_2.txt") as f:
    mywiki = f.read().splitlines() 

In [199]:
wolof1 = wolof + mywiki

In [200]:
sent = wolof5
len(sent)

58768

In [201]:
# corpus = ''
# for i in sen:
#     corpus = corpus + str(i) +'. '

### train/test split

In [202]:
training = sent[:int(len(sent)*0.8)]
testing = sent[int(len(sent)*0.8):]

In [203]:
training_text = []
for item in training:
    training_text.append(item.split())

testing_text = []
for item in testing:
    testing_text.append(item.split())

In [204]:
train, vocab = padded_everygram_pipeline(2, training_text)

### modeling

In [205]:
lm = MLE(2)
lm.fit(train, vocab)

### create bigrams for testing purpose

In [206]:
test_grams = []
for item in testing_text:
    test_grams.append(list(bigrams(' '.join(item).split()))) 
merged_test_grams = list(itertools.chain(*test_grams))

In [207]:
train_grams = []
for item in training_text:
    train_grams.append(list(bigrams(' '.join(item).split()))) 
merged_train_grams = list(itertools.chain(*train_grams))

In [208]:
print(len(merged_train_grams))
print(len(merged_test_grams))

696127
112457


In [209]:
train_gram_set = set(merged_train_grams)
test_gram_set = set(merged_test_grams)
print(len(train_gram_set))
print(len(test_gram_set))


199534
65647


In [210]:
intersection = train_gram_set.intersection(test_gram_set)
print(len(intersection))

51680


## perplexity

In [211]:
lm.perplexity(list(intersection))

91.22471300613424